In [17]:
from utils.xtb_density import CDCalculator, interplot_ecloud
from utils.grid import *
from utils.chem import *
from moleculekit.smallmol.smallmol import SmallMol
from moleculekit.tools.voxeldescriptors import _getOccupancyC

In [7]:
calculater = CDCalculator(xtb_command='/home/haotian/Molecule_Generation/MG/ECloudGen/xtb-bleed/bin/xtb')

In [5]:
pkt_mol = Chem.MolFromPDBFile('./pkt.pdb')
lig_mol = read_sdf('./lig.sdf')[0]

In [8]:
resolution = 0.3
size = 24
N = [size, size, size]
llc = (np.zeros(3) - float(size * 1. / 2))
# Now, the box is 24×24×24 A^3
expanded_pcenters = BuildGridCenters(llc, N, resolution)

In [14]:
rotation = True

pkt_smallmol = SmallMol(pkt_mol)
lig_smallmol = SmallMol(lig_mol)


lig_coords = lig_smallmol._coords[: , : , 0]
lig_center = np.mean(lig_coords, axis=0)

# define the pkt channel
pkt_sigmas, pkt_coords, pkt_center = generate_sigmas(pkt_smallmol)

# use the pkt_center as the whole center
pkt_grids = expanded_pcenters + pkt_center
lig_grids = expanded_pcenters + lig_center


# Do the rotation
if rotation:
    rrot = uniformRandomRotation()  # Rotation
    lig_coords = rotate(lig_coords, rrot, center=lig_center)
    pkt_coords = rotate(pkt_coords, rrot, center=pkt_center)



pkt_grid = _getOccupancyC(pkt_coords.astype(np.float32),
                            pkt_grids.reshape(-1, 3),
                            pkt_sigmas).reshape(size, size, size, 8)

lig_ecloud = calculater.calculate(lig_mol)

2023-05-18 10:56:55,862 - moleculekit.smallmol.smallmol - INFO - Adding any missing hydrogens (fixHs=True)
2023-05-18 10:56:55,863 - moleculekit.smallmol.smallmol - INFO - Adding any missing hydrogens (fixHs=True)


In [32]:
lig_density = interplot_ecloud(lig_ecloud, expanded_pcenters.transpose(3,0,1,2)).reshape(N)